In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [16]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f'Using device: {device}')
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print(f'Using device: {device}')

Using device: cuda
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [17]:
# Load data
df = pd.read_csv("BMED_DB_augmented.csv")
df = df[df['exp'].isin([0])].reset_index(drop=True)

In [18]:
# Robust min-max scaling including safety margin
ranges ={
'V' : {'min':0, 'max':50},
'E' : {'min':0, 'max':1},
'VF' : {'min':0, 'max':2},
'VA' : {'min':0, 'max':2},
'VB' : {'min':0, 'max':8},
'CF_LA' : {'min':-1, 'max':4},
'CA_LA' : {'min':-1, 'max':4},
'CF_K' : {'min':-1, 'max':7},
'CB_K' : {'min':-1, 'max':2},
'I' : {'min':0, 'max':5},
}

In [19]:
# Data normalization
ndf = pd.DataFrame()
ndf['exp'] = df['exp']; ndf['t'] = df['t']

for col in ['V', 'E', 'VF', 'VA', 'VB', 'CF_LA', 'CA_LA', 'CF_K', 'CB_K', 'I']:
    if col in ranges:
        ndf[col] = (df[col] - ranges[col]['min'])/(ranges[col]['max'] - ranges[col]['min'])
    else:
        ndf[col] = df[col]

In [20]:
# prepare data
def prepare_data(ndf):
    '''
    prepare data list for each experiment

    Args:
        ndf: normalized dataframe
    
    Returns:
        Vt_list: list of applied voltage
        E_list: list of external electrolyte concentration
        CFLA_list: list of feed LA concentration
        CALA_list: list of acid LA concentration
        CFK_list: list of feed K concentration
        CBK_list: list of base K concentration
        VF_list: list of feed volume
        VA_list: list of acid volume
        VB_list: list of base volume
        I_list: list of current
    '''
    Vt_list, E_list, CFLA_list, CALA_list, CFK_list, CBK_list, VF_list, VA_list, VB_list, I_list = [], [], [], [], [], [], [], [], [], []

    for exp_num in ndf['exp'].unique():
        exp_data = ndf[ndf['exp'] == exp_num]

        # operating conditions
        Vt_list.append(exp_data['V'].values)
        E_list.append(exp_data['E'].values)

        # concentrations
        CFLA_list.append(exp_data['CF_LA'].values)
        CALA_list.append(exp_data['CA_LA'].values)
        CFK_list.append(exp_data['CF_K'].values)
        CBK_list.append(exp_data['CB_K'].values)

        # volumes
        VF_list.append(exp_data['VF'].values)
        VA_list.append(exp_data['VA'].values)
        VB_list.append(exp_data['VB'].values)

        # current
        I_list.append(exp_data['I'].values)

    return Vt_list, E_list, CFLA_list, CALA_list, CFK_list, CBK_list, VF_list, VA_list, VB_list, I_list

Vt_list, E_list, CFLA_list, CALA_list, CFK_list, CBK_list, VF_list, VA_list, VB_list, I_list = prepare_data(ndf)

In [21]:
# Pad sequences
def pad_sequences(data_list, max_length=None, pad_value=-100.0):
    '''
    Pad variables length sequences to the same length

    Args:
        data_list: list of tensors with different sequence lengths
        max_length: maximum length to pad to (default: longest sequence)
        pad_value: value to use for padding

    Returns:
        padded_tensor: [batch_size, max_length, ...] - padded sequences
        seq_lengths: [batch_size] - original sequence lengths
    '''

    if max_length is None:
        max_length = max(data.shape[0] for data in data_list) # Auto-calculate the max length
    
    batch_size = len(data_list) # Batch size
    seq_lengths = torch.tensor([data.shape[0] for data in data_list]) # Actual sequential length for each experiments
    dimensions = data_list[0].shape[1:] # Get shape of individual elements
    padded_tensor = torch.full((batch_size, max_length) + dimensions, pad_value, dtype=torch.float32) # generaste padded tensor filled with pad_value

    # Fill with actual data
    for i, data in enumerate(data_list):
        padded_tensor[i, :data.shape[0]] = torch.tensor(data[:data.shape[0]], dtype=torch.float32)
    
    return padded_tensor, seq_lengths, max_length

Vt, seq_lengths, max_length = pad_sequences(Vt_list)
E, _, _ = pad_sequences(E_list,max_length = max_length)
CFLA, _, _ = pad_sequences(CFLA_list,max_length = max_length)
CALA, _, _ = pad_sequences(CALA_list,max_length = max_length)
CFK, _, _ = pad_sequences(CFK_list,max_length = max_length)
CBK, _, _ = pad_sequences(CBK_list,max_length = max_length)
VF, _, _ = pad_sequences(VF_list,max_length = max_length)
VA, _, _ = pad_sequences(VA_list,max_length = max_length)
VB, _, _ = pad_sequences(VB_list,max_length = max_length)
I, _, _ = pad_sequences(I_list,max_length = max_length)

In [22]:
# Prepare input tensor
def prepare_input(Vt, E, CFLA, CALA, CFK, CBK, VF, VA, VB, seq_lengths):
    '''
    prepare input tensor for the model with padding support

    Args:
        Vt: applied voltage
        E: external electrolyte concentration
        CFLA: feed LA concentration
        CALA: acid LA concentration
        CFK: feed K concentration
        CBK: base K concentration
        VF: feed volume
        VA: acid volume
        VB: base volume
        seq_lengths: actual sequence lengths

    Returns:
        input_tensor: [batch_size, seq_len, 3, 6] - formatted input for CNN-LSTM
        initial_state: [batch_size, 3, 3] - initial concentrations and volumes
        mask: [batch_size, seq_len] - padding mask
        seq_lengths: [batch_size] - actual sequence lengths
    '''

    batch_size, seq_len = Vt.shape # Get batch size and sequence length for set the size of input tensor
    input = torch.zeros(batch_size, seq_len, 9) # Generate input tensor

    # Fill input tensor for each channel
    input[:, :, 0] = Vt # Applied voltage
    input[:, :, 1] = E # External electrolyte concentration
    input[:, :, 2] = CFLA # Feed LA concentration
    input[:, :, 3] = CALA # Acid LA concentration
    input[:, :, 4] = CFK # Feed K concentration
    input[:, :, 5] = CBK # Base K concentration
    input[:, :, 6] = VF # Feed volume
    input[:, :, 7] = VA # Acid volume
    input[:, :, 8] = VB # Base volume

    # initial state for each feature
    init = torch.zeros(batch_size, 9)
    init[:, 0] = Vt[:, 0] # Initial applied voltage
    init[:, 1] = E[:, 0] # Initial external electrolyte concentration
    init[:, 2] = CFLA[:, 0] # Initial feed LA concentration
    init[:, 3] = CALA[:, 0] # Initial acid LA concentration
    init[:, 4] = CFK[:, 0] # Initial feed K concentration
    init[:, 5] = CBK[:, 0] # Initial base K concentration
    init[:, 6] = VF[:, 0] # Initial feed volume
    init[:, 7] = VA[:, 0] # Initial acid volume
    init[:, 8] = VB[:, 0] # Initial base volume

    # Create padding mask
    mask = torch.zeros(batch_size, seq_len)
    for i, length in enumerate(seq_lengths):
        mask[i, :length] = 1.0

    return input, init, mask, seq_lengths

input_tensor, init, mask, seq_lengths = prepare_input(Vt, E, CFLA, CALA, CFK, CBK, VF, VA, VB, seq_lengths)

In [23]:
# Generate Dataset by experiments
class BMEDDataset(Dataset):
    def __init__(self, inputs, init, masks, seq_lengths, I_exp, CFLA_exp, CALA_exp, CFK_exp, CBK_exp, VF_exp, VA_exp, VB_exp):
        self.CV = inputs[:, :, :2] # Extract [Vt, E]
        self.init = init[:, 2:] # Extract state variables
        self.masks = masks
        self.seq_lengths = seq_lengths

        self.states = torch.stack([CFLA_exp, CALA_exp, CFK_exp, CBK_exp, VF_exp, VA_exp, VB_exp], dim=2)
        self.current = I_exp.unsqueeze(2) # [batch, seq, 1]

    def __len__(self):
        return len(self.CV)
    
    def __getitem__(self, idx):
        return {
            'CV': self.CV[idx],
            'init': self.init[idx],
            'masks': self.masks[idx],
            'seq_len': self.seq_lengths[idx],
            'states': self.states[idx],
            'current': self.current[idx]
        }
train_dataset = BMEDDataset(input_tensor, init, mask, seq_lengths, I, CFLA, CALA, CFK, CBK, VF, VA, VB)

In [24]:
# Generate DataLoader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [25]:
# Model Initialization
class BMEDModel(nn.Module):
    def __init__(self, hidden_nodes = 64, num_rnn_layers = 2, num_fnn_layers = 2,max_len = 37, dt = 0.25):
        super(BMEDModel, self).__init__()

        self.max_len = max_len
        self.input_features = 9 # [Vt, E, CFLA, CALA, CFK, CBK, VF, VA, VB]
        self.control_features = 2 # [Vt, E]
        self.state_features = 7 # [CFLA, CALA, CFK, CBK, VF, VA, VB]
        self.flux_features = 4 # [dLA, dK, dH2O_A, dH2O_B]
        self.current_features = 1 # [I]
        self.dt = dt # time step
        self.hidden_nodes = hidden_nodes
        self.num_rnn_layers = num_rnn_layers
        self.ranges = {
            'CFLA': {'min': -1, 'max': 4},
            'CALA': {'min': -1, 'max': 4}, 
            'CFK': {'min': -1, 'max': 7},
            'CBK': {'min': -1, 'max': 2},
            'VF': {'min': 0, 'max': 2},
            'VA': {'min': 0, 'max': 2},
            'VB': {'min': 0, 'max': 8}
        }


        # Layer Normalization
        self.layer_norm = nn.LayerNorm(self.input_features)

        # RNN layers
        self.rnn_layers = nn.LSTM(
            input_size = self.input_features,
            hidden_size = hidden_nodes,
            num_layers = num_rnn_layers,
            batch_first = True,
            dropout = 0.2 if num_rnn_layers > 1 else 0
        )

        # Flux Head
        flux_layers = []
        flux_sizes = [hidden_nodes]
        flux_step = (hidden_nodes - self.flux_features) / (num_fnn_layers)
        
        for i in range(num_fnn_layers):
            next_size = int(hidden_nodes - flux_step * (i + 1))
            if i == num_fnn_layers - 1:
                next_size = self.flux_features
            
            flux_layers.append(nn.Linear(flux_sizes[-1], next_size))
            flux_layers.append(nn.ELU())
            flux_layers.append(nn.Dropout(0.2))
            flux_sizes.append(next_size)
            
        self.flux_NN = nn.Sequential(*flux_layers)
        

        # Current Head
        current_layers = []
        current_sizes = [hidden_nodes]
        current_step = (hidden_nodes - self.current_features) / (num_fnn_layers)
        
        for i in range(num_fnn_layers):
            next_size = int(hidden_nodes - current_step * (i + 1))
            if i == num_fnn_layers - 1:
                next_size = self.current_features
            
            current_layers.append(nn.Linear(current_sizes[-1], next_size))
            current_layers.append(nn.ELU())
            current_layers.append(nn.Dropout(0.2))
            current_sizes.append(next_size)
        
        self.current_NN = nn.Sequential(*current_layers)
    
    def init_hidden(self, batch_size, device):
        """
        Initialize hidden states for RNN layers
        """
        h0 = torch.zeros(self.num_rnn_layers, batch_size, self.hidden_nodes, device=device)
        c0 = torch.zeros(self.num_rnn_layers, batch_size, self.hidden_nodes, device=device)
        return (h0, c0)
    
    def denormalize_state(self, norm_state):
        '''
        Normalized state to real values 
        '''
        batch_size = norm_state.shape[0]
        real_state = torch.zeros_like(norm_state)

        state_names = ['CFLA', 'CALA', 'CFK', 'CBK', 'VF', 'VA', 'VB']

        for i, name in enumerate(state_names):
            min_val = self.ranges[name]['min']
            max_val = self.ranges[name]['max']
            real_state[:, i] = norm_state[:, i] * (max_val - min_val) + min_val
        
        return real_state
    
    def normalize_state(self, real_state):
        '''
        Real state to normalized values
        '''
        batch_size = real_state.shape[0]
        norm_state = torch.zeros_like(real_state)

        state_names = ['CFLA', 'CALA', 'CFK', 'CBK', 'VF', 'VA', 'VB']

        for i, name in enumerate(state_names):
            min_val = self.ranges[name]['min']
            max_val = self.ranges[name]['max']
            norm_state[:, i] = (real_state[:, i] - min_val) / (max_val - min_val)

        return norm_state

    def MB_step(self, norm_state, LA_flux, K_flux, VFA_flux, VFB_flux):
        '''
        Perform one time step of mass balance
        state: [batch, 7] - [CFLA, CALA, CFK, CBK, VF, VA, VB]
        '''
        real_state = self.denormalize_state(norm_state)

        # extract current values
        # channel 0: feed, channel 1: acid, channel 2: base
        # property 0: LA_conc, property 1: K_conc, property 2: volume

        CFLA = real_state[:, 0]
        CALA = real_state[:, 1]
        CFK = real_state[:, 2]
        CBK = real_state[:, 3]
        VF = real_state[:, 4]
        VA = real_state[:, 5]
        VB = real_state[:, 6]

        # volume changes due to water flux
        # Assuming positive flux means water moves from feed to acid or base
        nVF = VF - (VFA_flux + VFB_flux) * self.dt # Feed Volume
        nVA = VA + VFA_flux * self.dt # Acid Volume
        nVB = VB + VFB_flux * self.dt # Base Volume

        # LA mass balance
        nNFLA = CFLA*VF - LA_flux*self.dt
        nNALA = CALA*VA + LA_flux*self.dt
        nNFK = CFK*VF - K_flux*self.dt
        nNBK = CBK*VB + K_flux*self.dt

        # update states
        new_real_state = torch.zeros_like(real_state)
        new_real_state[:, 0] = nNFLA / (nVF + 1e-8) # new Feed LA concentration
        new_real_state[:, 1] = nNALA / (nVA + 1e-8) # new Acid LA concentration
        new_real_state[:, 2] = nNFK / (nVF + 1e-8) # new Feed K concentration
        new_real_state[:, 3] = nNBK / (nVB + 1e-8) # new Base K concentration
        new_real_state[:, 4] = nVF # new Feed Volume
        new_real_state[:, 5] = nVA # new Acid Volume
        new_real_state[:, 6] = nVB # new Base Volume

        new_norm_state = self.normalize_state(new_real_state)
    
        return new_norm_state
    
    def forward_single_step(self, CV, prev_state, hidden_state):
        '''
        Predict Single Step of BMED
        CV: [batch_size, 2] - [Vt, E]
        prev_state: [batch_size, 7] - [CFLA, CALA, CFK, CBK, VF, VA, VB]
        hidden_state: [h, c] - hidden state of RNN layers
        '''

        full_input = torch.cat([CV, prev_state], dim=1)
        full_input = full_input.unsqueeze(1) # [batch_size, 1, 9]

        # Layer Normalization
        rnn_input = self.layer_norm(full_input)
        
        # RNN forward - pack all information of previous hidden state
        rnn_out, new_hidden_state = self.rnn_layers(rnn_input, hidden_state)

        # Predict flux and current
        flux = self.flux_NN(rnn_out.squeeze(1)) # [batch_size, 4]
        current = self.current_NN(rnn_out.squeeze(1)) # [batch_size, 1]

        # Physical update
        LA_flux = flux[:, 0]
        K_flux = flux[:, 1]
        VFA_flux = flux[:, 2]
        VFB_flux = flux[:, 3]

        new_state = self.MB_step(prev_state, LA_flux, K_flux, VFA_flux, VFB_flux)

        return current, new_state, new_hidden_state
    
    def next_seq_pred_train(self, CV, init, states, current, masks):
        '''
        Autoregressive training
        '''
        batch_size, max_seq_len, _ = CV.shape
        device= CV.device

        mask = masks # [batch_size, max_seq_len]

        CVf = CV[:, 0, :] # [batch_size, 2]

        # Initialize hidden state
        hidden_state = self.init_hidden(batch_size,device)
        current_state = init.clone() # [batch_size, 7]

        # predicted results
        predicted_current = []
        predicted_states = []

        # use init_state at t=0
        predicted_states.append(init)

        # use 0 at t=0
        first_current = torch.zeros(batch_size, 1, device=device)
        predicted_current.append(first_current)

        # Calculate for each timestep with the entire batch simultaneously
        for t in range(1,max_seq_len):

            # Predict current and states of the entire batch
            pred_current, pred_state, hidden_state = self.forward_single_step(CVf, current_state, hidden_state)

            predicted_current.append(pred_current)
            predicted_states.append(pred_state)

            # update current state with predicted states
            current_state = pred_state

        # Convert to tensor
        predicted_current = torch.stack(predicted_current, dim=1) # [batch_size, max_seq_len, 1]
        predicted_states = torch.stack(predicted_states, dim=1) # [batch_size, max_seq_len, 7]

        # Loss calculation: 
        states_exp = states[:, 1:, :] # [batch_size, seq_len-1, 7]
        current_exp = current[:, 1:, :] # [batch_size, seq_len-1, 1]
        states_pred = predicted_states[:, 1:, :] # [batch_size, seq_len-1, 7]
        current_pred = predicted_current[:, 1:, :] # [batch_size, seq_len-1, 1]
        mask_adj = mask[:, 1:] # [batch_size, seq_len-1]
        
        # Calculate loss
        states_loss = F.mse_loss(states_pred, states_exp, reduction='none')
        current_loss = F.mse_loss(current_pred, current_exp, reduction='none')

        # Apply mask and calculate mean loss
        mask_states = mask_adj.unsqueeze(-1).expand(-1, -1, 7)
        mask_current = mask_adj.unsqueeze(-1)

        masked_states_loss = (states_loss * mask_states).sum() / mask_states.sum()
        masked_current_loss = (current_loss * mask_current).sum() / mask_current.sum()

        total_loss = masked_current_loss + masked_states_loss*7

        return total_loss, predicted_current, predicted_states
    
    def generate_sequence(self, CV, init, max_steps):
        '''
        Prediction mode from initial state

        Args:
            CV: [2] or [batch_size, 2] - [Vt, E]
            init_state: initial state ([7] or [batch_size, 7])
            max_steps: maximum number of steps to generate
        '''

        if len(init.shape) == 1:
            # if input is single exp, add a batch dimension
            init = init.unsqueeze(0)
            batch_size = 1
            single_exp = True
        else:
            batch_size = init.shape[0]
            single_exp = False
        
        device = init.device

        # check CV batch size and modification
        if len(CV.shape) == 1:
            CV = CV.unsqueeze(0) # [1, 2]
        
        CVf = CV[:, 0, :].to(device)
        

        # initialize hidden state
        hidden_state = self.init_hidden(batch_size, device)
        current_state = init.clone() # [batch_size, 7]

        predicted_current = []
        predicted_states = []
        
        predicted_states.append(init)
        first_current = torch.zeros(batch_size, 1, device=device)  # 첫 번째 전류는 0으로 설정
        predicted_current.append(first_current)

        for t in range(1, max_steps):
            pred_current, pred_state, hidden_state = self.forward_single_step(CVf, current_state, hidden_state)

            predicted_current.append(pred_current) # [batch_size, 1]
            predicted_states.append(pred_state) # [batch_size, 7]

            current_state = pred_state

        predicted_current = torch.stack(predicted_current, dim=1) # [batch_size, max_steps, 1]
        predicted_states = torch.stack(predicted_states, dim=1) # [batch_size, max_steps, 7]

        if single_exp:
            predicted_current = predicted_current.squeeze(0) # [max_steps, 1]
            predicted_states = predicted_states.squeeze(0) # [max_steps, 7]
        
        return predicted_current, predicted_states

    def forward(self, CV, init, states, current, masks, mode='train', max_steps=None):
        '''
        Integrated forward method
        '''

        if mode == 'train':
            if states is None or current is None or masks is None:
                raise ValueError("states, current, and masks are required for training")
            return self.next_seq_pred_train(CV, init, states, current, masks)
        elif mode == 'inference':
            if max_steps is None:
                raise ValueError("max_steps is required for inference")
            fixed_control = CV[:, 0, :] # [batch_size, 2]
            return self.generate_sequence(CV, init, max_steps)
        else:
            raise ValueError("Invalid mode. Use 'train' or 'inference'")
dmodel = 128
model = BMEDModel(
    hidden_nodes=dmodel,
    num_rnn_layers=10,
    num_fnn_layers=5,
    max_len=max_length,
    dt=0.25
)
model.to(device)

BMEDModel(
  (layer_norm): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
  (rnn_layers): LSTM(9, 128, num_layers=10, batch_first=True, dropout=0.2)
  (flux_NN): Sequential(
    (0): Linear(in_features=128, out_features=103, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=103, out_features=78, bias=True)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=78, out_features=53, bias=True)
    (7): ELU(alpha=1.0)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=53, out_features=28, bias=True)
    (10): ELU(alpha=1.0)
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=28, out_features=4, bias=True)
    (13): ELU(alpha=1.0)
    (14): Dropout(p=0.2, inplace=False)
  )
  (current_NN): Sequential(
    (0): Linear(in_features=128, out_features=102, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=102, out_featu

In [26]:
# Noam Scheduler
class NoamLR(torch.optim.lr_scheduler._LRScheduler):
    '''
    Pytorch LRScheduler 스타일으 Noam Scheduler
    '''

    def __init__(self, optimizer, d_model, warmup_steps=40, last_epoch=-1):
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        super(NoamLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        step = self.last_epoch + 1
        if step == 0:
            step  = 1

        lr_scale = (self.d_model ** -0.5) *min(
            step ** -0.5,
            step * (self.warmup_steps ** -1.5)
        )
        return [base_lr * lr_scale for base_lr in self.base_lrs]

In [27]:
# Training Configuration
num_epochs = 1000
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
scheduler = NoamLR(optimizer, d_model=dmodel, warmup_steps=0.1*num_epochs)

In [28]:

def train_epoch(model, train_loader, optimizer, scheduler, device):
    # Train
    model.train()
    total_loss = 0.0
    num_batches = 0
    best_loss = np.inf
    best_states = None
    best_current = None

    for batch in train_loader:
        # move to device
        CV = batch['CV'].to(device)
        init = batch['init'].to(device)
        states = batch['states'].to(device)
        current = batch['current'].to(device)
        masks = batch['masks'].to(device)

        # gradient initialization
        optimizer.zero_grad()

        # Forward pass
        loss, pred_current, pred_states = model(CV, init, states, current, masks, mode='train')

        # Backward pass
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm = 1)

        # Update parameters
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    # update scheduler
    scheduler.step()

    return total_loss / num_batches, pred_current, pred_states

print("Start training...")
print("=" * 50)

best_loss = np.inf
best_current = None
best_states = None

for epoch in range(num_epochs):
    train_loss, pred_current, pred_states = train_epoch(model, train_loader, optimizer, scheduler, device)

    current_lr = optimizer.param_groups[0]['lr']

    if train_loss < best_loss:
        best_loss = train_loss
        best_current = pred_current
        best_states = pred_states

    if epoch % 10 == 0: 
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.6f}, LR: {current_lr:.6f}, best_loss: {best_loss:.6f}")

Start training...
Epoch 1/1000, Loss: 0.085248, LR: 0.000018, best_loss: 0.085248
Epoch 11/1000, Loss: 0.058940, LR: 0.000106, best_loss: 0.058940
Epoch 21/1000, Loss: 0.042523, LR: 0.000194, best_loss: 0.035290
Epoch 31/1000, Loss: 0.013484, LR: 0.000283, best_loss: 0.013484
Epoch 41/1000, Loss: 0.010979, LR: 0.000371, best_loss: 0.010864
Epoch 51/1000, Loss: 0.011528, LR: 0.000460, best_loss: 0.006420
Epoch 61/1000, Loss: 0.010328, LR: 0.000548, best_loss: 0.005917
Epoch 71/1000, Loss: 0.006661, LR: 0.000636, best_loss: 0.005545
Epoch 81/1000, Loss: 0.010631, LR: 0.000725, best_loss: 0.004291
Epoch 91/1000, Loss: 0.008232, LR: 0.000813, best_loss: 0.004291
Epoch 101/1000, Loss: 0.004064, LR: 0.000875, best_loss: 0.003078
Epoch 111/1000, Loss: 0.004076, LR: 0.000835, best_loss: 0.002710
Epoch 121/1000, Loss: 0.002747, LR: 0.000800, best_loss: 0.002710
Epoch 131/1000, Loss: 0.005533, LR: 0.000769, best_loss: 0.002043
Epoch 141/1000, Loss: 0.005461, LR: 0.000742, best_loss: 0.002043
Epo

In [29]:
test_idx = 0
test_states = train_dataset.states[test_idx:test_idx+1]
test_current = train_dataset.current[test_idx:test_idx+1]
test_seq_len = train_dataset.seq_lengths[test_idx:test_idx+1]


: 

In [ ]:
pred_current_cpu = best_current.detach().cpu().numpy()
pred_states_cpu = best_states.detach().cpu().numpy()
test_states_cpu = test_states.detach().cpu().numpy()
test_current_cpu = test_current.detach().cpu().numpy()

plt.figure(figsize=(12, 6))
plt.plot(test_current_cpu[0, :test_seq_len[0], 0], label='True Current')
plt.plot(pred_current_cpu[0, :test_seq_len[0], 0], label='Predicted Current')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))

In [ ]:
plt.figure(figsize=(15, 12))  # 가로 크기를 늘려서 2열로 표시

# 첫 번째 열 (왼쪽)
plt.subplot(4, 2, 1)
plt.plot(test_states_cpu[0, :test_seq_len[0], 0], label='True CFLA')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 0], label='Predicted CFLA')
plt.legend()
plt.title('CFLA')

plt.subplot(4, 2, 3)
plt.plot(test_states_cpu[0, :test_seq_len[0], 1], label='True CALA')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 1], label='Predicted CALA')
plt.legend()
plt.title('CALA')

plt.subplot(4, 2, 5)
plt.plot(test_states_cpu[0, :test_seq_len[0], 2], label='True CFK')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 2], label='Predicted CFK')
plt.legend()
plt.title('CFK')

plt.subplot(4, 2, 7)
plt.plot(test_states_cpu[0, :test_seq_len[0], 3], label='True CBK')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 3], label='Predicted CBK')
plt.legend()
plt.title('CBK')

# 두 번째 열 (오른쪽)
plt.subplot(4, 2, 2)
plt.plot(test_states_cpu[0, :test_seq_len[0], 4], label='True VF')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 4], label='Predicted VF')
plt.legend()
plt.title('VF')

plt.subplot(4, 2, 4)
plt.plot(test_states_cpu[0, :test_seq_len[0], 5], label='True VA')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 5], label='Predicted VA')
plt.legend()
plt.title('VA')

plt.subplot(4, 2, 6)
plt.plot(test_states_cpu[0, :test_seq_len[0], 6], label='True VB')
plt.plot(pred_states_cpu[0, :test_seq_len[0], 6], label='Predicted VB')
plt.legend()
plt.title('VB')

plt.tight_layout()
plt.show()


In [ ]:
test_init